In [ ]:
import os
import fitz
import datetime
import platform
import time
from pptx import Presentation
from pptx.util import Pt,Inches
from tqdm import tqdm
# from PIL import Image
# from pdf2image import convert_from_path,convert_from_bytes
import tempfile
# from pdf2image.exceptions import (
#     PDFInfoNotInstalledError,
#     PDFPageCountError,
#     PDFSyntaxError
# )

class pdfToPPTX:
    def __init__(self,pdfname, imagePath, *pageNum):
        #涉及不同的操作系统，需转换
        thissys=platform.platform().lower() 
        if 'windows' in thissys:
            print("windows 系统 \n")
            self.pdfname=pdfname.replace("/","\\")
            self.imagePath=imagePath.replace("/","\\")
        elif 'linux' in thissys:
            print("linux 系统 \n")
            self.pdfname=pdfname.replace("\\","\/")
            self.imagePath=imagePath.replace("\\","\/")                                
        self.pagenum=pageNum
        
    def pdf2image(self):
        pgns=self.outpageNum()

        
        with tempfile.TemporaryDirectory() as path:
            print("正在读取Pdf文件…… \n")
            images_from_path = convert_from_path(self.pdfname)
            print("文件总页数 {} 页。 \n".format(len(images_from_path)))
            print("正在转换…… \n")   
            time.sleep(0.5)
            for pgn in tqdm(pgns):
                if not os.path.exists(self.imagePath):
                    os.makedirs(self.imagePath)
                images_from_path[int(pgn)].save(self.imagePath+'/'+'psReport_%s.jpg' % str(int(pgn+1)).zfill(3),'jpeg')
#             print(images_from_path)
        print("done")
    
    def image_adj(self,w,h):
        
        print("正在调整图片大小至{0}x{1}... \n".format(w,h))
        for pic in os.listdir(self.imagePath):            
            if pic[-3:].lower()=="jpg":
                _img=Image.open(os.path.join(self.imagePath,pic))
                img=_img.resize((w,h))
                img.save(os.path.join(self.imagePath,pic))
        print("done \n")                                                

    def outpageNum(self):
        pgn=[]
        
        if isinstance(self.pagenum[0],str):
            if "-" in self.pagenum[0]:
                firstpage=self.pagenum[0][:self.pagenum[0].find("-")]
                lastpage=self.pagenum[0][self.pagenum[0].find("-")+1:]
                for i in range(int(firstpage)-1,int(lastpage)-1):
                    pgn.append(i)
                pgn.append(int(lastpage)-1) #包括最后一页
            else:
                pass
        else:
            pgn=[int(x)-1 for x in self.pagenum]

        return pgn
    
    def pyMuPDF_fitz(self):
        pgns=self.outpageNum()
        
        startTime_pdf2img = datetime.datetime.now()#开始时间
        print("imagePath="+self.imagePath)        
        
        pdfDoc = fitz.open(self.pdfname)
        
            
        totalPageNum=pdfDoc.pageCount
        
        for pg in pgns:
            page = pdfDoc[pg]
            rotate = int(0)
            # 每个尺寸的缩放系数为1.3，这将为我们生成分辨率提高2.6的图像。
            # 此处若是不做设置，默认图片大小为：792X612, dpi=96
            zoom_x = 1.33333333 #(1.33333333-->1056x816)   (2-->1584x1224)
            zoom_y = 1.33333333
            mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
            pix = page.getPixmap(matrix=mat, alpha=False)

            pix.writePNG(self.imagePath+'/'+'images_%s.png' % str(pg+1).zfill(3))#将图片写入指定的文件夹内

        endTime_pdf2img = datetime.datetime.now()#结束时间
        print('pdf2img时间=',(endTime_pdf2img - startTime_pdf2img).seconds)
        
    def toPPT(self):
        prs = Presentation()
        prs.slide_height = 6858000    #设置ppt的高度
        prs.slide_width = 12192000    #设置ppt的宽度
        
        for pic in os.listdir(self.imagePath):
            if pic[-3:].lower()=='png':

                img_path = os.path.join(self.imagePath,pic)  # 文件路径

                # 改图片大小
#                 print(os.path.join(self.p,pic))
                # _img=Image.open(img_path)
                # ht,wt=_img.size
                # img=_img.resize((1024,int(1024*wt/ht)))
                # img.save(img_path)               
                
                left, top, width, height = Inches(1), Inches(0.5), Inches(11.5), Inches(2)  # 预设位置及大小  
                slide = prs.slides.add_slide(prs.slide_layouts[5]) #prs.slide_layouts中一共预存有1-48种，采用第6六种为空白幻灯片
                
                body_shape = slide.shapes.placeholders  # body_shape为本页ppt中所有shapes
                body_shape[0].text = 'this is placeholders[0]'  # 在第一个文本框中文字框架内添加文字
                
                pic = slide.shapes.add_picture(img_path, left, top,width)  # 在指定位置按预设值添加图
                slide.shapes._spTree.insert(1, pic._element)
                
                
        prs.save(os.path.join(self.imagePath,'pic.pptx'))
        print("done")
        
        
    
if __name__=="__main__":
    pdf='/home/luo/data/大智小超/乐高/图纸/100份wedo2.0图纸.pdf'
    jpg='/home/luo/data/大智小超/乐高/图纸/output_pdf'
    pdf=pdfToPPTX(pdf,jpg,6,9)
#     pdf.pdf2image()
#     pdf.pyMuPDF_fitz()
    pdf.toPPT()
#     pdf.image_adj(1024,576)

In [ ]:
import sys, fitz
import os
import datetime
 
def pyMuPDF_fitz(pdfPath, imagePath):
    startTime_pdf2img = datetime.datetime.now()#开始时间
    
    print("imagePath="+imagePath)
    pdfDoc = fitz.open(pdfPath)
    for pg in range(pdfDoc.pageCount):
        page = pdfDoc[pg]
        rotate = int(0)
        # 每个尺寸的缩放系数为1.3，这将为我们生成分辨率提高2.6的图像。
        # 此处若是不做设置，默认图片大小为：792X612, dpi=96
        zoom_x = 1.33333333 #(1.33333333-->1056x816)   (2-->1584x1224)
        zoom_y = 1.33333333
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        
        if not os.path.exists(imagePath):#判断存放图片的文件夹是否存在
            os.makedirs(imagePath) # 若图片文件夹不存在就创建
        
        pix.writePNG(imagePath+'/'+'images_%s.png' % pg)#将图片写入指定的文件夹内
        
    endTime_pdf2img = datetime.datetime.now()#结束时间
    print('pdf2img时间=',(endTime_pdf2img - startTime_pdf2img).seconds)
 
 
if __name__ == "__main__":
    pdf='/home/luo/data/大智小超/乐高/图纸/100份wedo2.0图纸.pdf'
    jpg='/home/luo/data/大智小超/乐高/图纸/output_pdf'
    pyMuPDF_fitz(pdfPath, imagePath)

In [20]:
from PIL import Image

img_path="/home/luo/data/大智小超/乐高/图纸/output_pdf/images_006.png"

img=Image.open(img_path)
w,h=img.size
img2=img.crop((0,35,w,35+w/1.77))
img2.save("/home/luo/data/大智小超/乐高/图纸/output_pdf/img2.png")

In [ ]:
import os
from pptx import Presentation
from pptx.util import Pt,Inches

def toppt():
    imgpath="/home/jack/文档/大智小超/output_pdf"
    prs = Presentation()
    prs.slide_height = 6858000    #设置ppt的高度
    prs.slide_width = 12192000    #设置ppt的宽度
        
    
    for pic in os.listdir(imgpath):
        if pic[-3:].lower()=="png":
            print(os.path.join(imgpath,pic))   
            _pic=os.path.join(imgpath,pic)

            left, top, width, height = Inches(1), Inches(0.5), Inches(11.5), Inches(2)  # 预设位置及大小  
            slide = prs.slides.add_slide(prs.slide_layouts[5]) #prs.slide_layouts中一共预存有1-48种，采用第6六种为空白幻灯片

            body_shape = slide.shapes.placeholders  # body_shape为本页ppt中所有shapes
            body_shape[0].text = '这个是测试标题'  # 在第一个文本框中文字框架内添加文字


            pic = slide.shapes.add_picture(_pic, left, top,width)  # 在指定位置按预设值添加图
            slide.shapes._spTree.insert(1, pic._element)   
            
            left = top = width = height = Inches(5)  # 预设位置及大小
            textbox = slide.shapes.add_textbox(left, top, width, height)  # left，top为相对位置，width，height为文本框大小
            textbox.text = 'this is a new textbox'  # 文本框中文字
            new_para = textbox.text_frame.add_paragraph()  # 在新文本框中添加段落
            new_para.text = 'this is second para in textbox'  # 段落文字


            prs.save(os.path.join(imgpath,'pic.pptx'))

toppt()